In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
import riskfolio as rp
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2016-01-01'
end = '2019-12-30'

# Tickers of assets
assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
assets.sort()

# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets

[*********************100%%**********************]  25 of 25 completed


In [3]:
# Calculating returns

Y = data[assets].pct_change().dropna()

display(Y.head())

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,-2.0256%,0.4057%,0.4036%,1.9692%,0.0180%,0.9305%,0.3678%,0.5783%,0.9482%,-1.1953%,...,1.5881%,0.0212%,2.8236%,0.9758%,0.6987%,1.7539%,-0.1730%,0.2410%,1.3734%,-1.0857%
2016-01-06,-11.4864%,-1.5879%,0.2411%,-1.7557%,-0.7727%,-1.2473%,-0.1736%,-1.1239%,-3.5866%,-0.9551%,...,0.5547%,0.0212%,0.1592%,-1.5646%,0.3108%,-1.0155%,-0.7653%,-3.0048%,-0.9035%,-2.9145%
2016-01-07,-5.1388%,-4.1922%,-1.6573%,-2.7699%,-1.1047%,-1.9769%,-1.2207%,-0.8855%,-4.6059%,-2.5394%,...,-2.2066%,-3.0310%,-1.0411%,-3.1557%,-1.6148%,-0.2700%,-2.2844%,-2.0570%,-0.5492%,-3.0019%
2016-01-08,0.2736%,-2.2705%,-1.6037%,-2.5425%,0.1099%,-0.2241%,0.5706%,-1.6402%,-1.7642%,-0.1649%,...,-0.1539%,-1.1366%,-0.7307%,-0.1448%,0.0895%,-3.3839%,-0.1117%,-1.1387%,-0.9719%,-1.1254%
2016-01-11,-4.3383%,0.1692%,-1.6850%,-1.0216%,0.0915%,-1.1791%,0.5674%,0.5287%,0.6616%,0.0330%,...,1.6436%,0.0000%,0.9869%,-0.1450%,1.2224%,1.4570%,0.5366%,-0.4607%,0.5800%,-1.9919%


In [4]:
def calculate_portfolio_returns(daily_returns, weights):
    weighted_returns = daily_returns * weights.values
    portfolio_return = weighted_returns.sum(axis=1)
    cumulative_return = (1 + portfolio_return).cumprod() - 1
    df_returns = pd.DataFrame({
        'Daily Return': portfolio_return,
        'Cumulative Return': cumulative_return
    })
    return df_returns


In [5]:
port = rp.Portfolio(returns=Y)

# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
weights,0.0000%,6.1590%,11.5018%,0.0000%,0.0000%,8.4805%,0.0000%,3.8193%,0.0000%,0.0000%,...,10.8262%,0.0000%,0.0000%,0.0000%,0.0000%,7.1804%,0.0001%,0.0000%,4.2737%,0.0000%


In [6]:
risk = ['MV', 'CVaR', 'MDD']
label = ['Std. Dev.', 'CVaR', 'Max Drawdown']
alpha = 0.05

for i in range(3):
    limits = port.frontier_limits(model=model, rm=risk[i], rf=rf, hist=hist)
    risk_min = rp.Sharpe_Risk(limits['w_min'], cov=cov, returns=returns, rm=risk[i], rf=rf, alpha=alpha)
    risk_max = rp.Sharpe_Risk(limits['w_max'], cov=cov, returns=returns, rm=risk[i], rf=rf, alpha=alpha)

    if 'Drawdown' in label[i]:
        factor = 1
    else:
        factor = 252**0.5

    print('\nMin Return ' + label[i] + ': ', (mu @ limits['w_min']).item() * 252)
    print('Max Return ' + label[i] + ': ',  (mu @ limits['w_max']).item() * 252)
    print('Min ' + label[i] + ': ', risk_min * factor)
    print('Max ' + label[i] + ': ', risk_max * factor)

NameError: name 'cov' is not defined

In [ ]:
rm = 'CVaR' # Risk measure
obj = 'Sharpe' # Objective function
# Constraint on maximum CVaR
l = 1 #

port.upperCVaR = 0.24/252**0.5 # We transform annual CVaR to daily CVaR

#rm = 'MV' obj = 'MinRisk' 2)
w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

In [ ]:
ax = rp.plot_pie(w=w, title='Sharpe Mean CVaR', others=0.05, nrow=25, cmap = "tab20",
                 height=6, width=10, ax=None)

In [ ]:
port_test = calculate_portfolio_returns(Y, w.T)
stock = port_test['Daily Return']
df_new = pd.DataFrame((stock.sharpe(),
    stock.cvar(),
    stock.value_at_risk(),
    stock.var(),
    stock.volatility()))
df_new = df_new.T
df_new.columns = ['Sharpe', 'CVaR', 'VaR', 'Var', 'Volatility']
df = pd.concat([df, df_new])



In [ ]:
df

In [ ]:
points = 50 # Number of points of the frontier

frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

display(frontier.T.head())

ax = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm='CVaR',
                      rf=rf, alpha=0.05, cmap='viridis', w=w, label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)

In [105]:

class PortfolioOptimizerTest:
    """
    This class is used to TEST optimize the portfolio using the risk-folio library
    """
    method_mu = 'hist'
    method_cov = 'hist'
    model = 'Classic'
    rm = 'CVaR'
    hist = True
    rf = 0
    lib = 0
    OBJ_OPTIONS = {'MinRisk', 'Utility', 'Sharpe', 'MaxRet'}
    RISK_LEVEL_USER = {'Conservative': 1, 'Moderate': 2, 'Aggressive': 3, 'Very Aggressive': 4}
    RISK_LEVEL_MAPPING = {
        1: 'MinRisk',
        2: 'Utility',
        3: 'Sharpe',
        4: 'MaxRet'
    }
    RISK_LEVEL_DEFAULT = 'Moderate'
    
    def __init__(self):
        self.weights_initial_sum = None
        self._assets = None
        self._cvar_value = None
        self._risk_level = None
        self._obj = 'Sharpe'
    
    @property
    def obj(self):
        return self._obj
    @property
    def cvar_value(self):
        return self._cvar_value
    
    @property
    def risk_level(self):
        return self._risk_level
    
    @cvar_value.setter
    def cvar_value(self, cvar_value):
        self._cvar_value = cvar_value / 252 ** 0.5

    @risk_level.setter
    def risk_level(self, risk_profile):
        self._risk_level = self.RISK_LEVEL_USER.get(risk_profile, self.RISK_LEVEL_DEFAULT)
        self.set_objective_function(self.RISK_LEVEL_MAPPING.get(self._risk_level, self.RISK_LEVEL_DEFAULT))

    @property
    def assets(self):
        return self._assets

    @property
    def weights_initial_sum(self):
        return self._weights_initial_sum

    @weights_initial_sum.setter
    def weights_initial_sum(self, value):
        self._weights_initial_sum = value

    def set_initial_asset(self, assets, weights=None):
        if not isinstance(assets, (list, tuple)):
            raise TypeError("The 'assets' parameter must be a list or tuple.")

        if weights is not None and not isinstance(weights, (list, tuple)):
            raise TypeError("The 'weights' parameter must be a list or tuple.")

        if weights is not None and len(assets) != len(weights):
            raise ValueError("The lengths of 'assets' and 'weights' must be the same.")

        self._assets = pd.DataFrame({'assets': assets, 'weights': weights})

        weights_initial_sum = self._assets['weights'].sum()
        self.weights_initial_sum = weights_initial_sum
        print('Initial weights sum:', weights_initial_sum)




    def set_objective_function(self, objective: str):
        """
        Set the objective function for portfolio optimization.
        :param objective: str
            The name of the objective function (e.g., 'MinRisk', 'Utility', 'Sharpe', 'MaxRet').
        """
        if objective in self.OBJ_OPTIONS:
            self._obj = objective
        else:
            raise ValueError(f'Invalid objective function: {objective}')

 
    def port_optimize(self, returns_train):
        portfolio = rp.Portfolio(returns=returns_train)
        portfolio.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
        if self.cvar_value is not None:
            try:
                portfolio.upperCVaR = self.cvar_value
            except Exception as e:
                print(e)
        weights_optimized = portfolio.optimization(
            model=self.model,
            rm=self.rm,
            obj=self.obj,
            rf=self.rf,
            l=self.lib,
            hist=True)
        if self.weights_initial_sum is not None:
            try:
                weights_optimized['weights'] = weights_optimized['weights'] * (1 - self.weights_initial_sum)
                self.assets.set_index('assets', inplace=True)
                weights_optimized = pd.concat([self.assets, weights_optimized], axis=0)
            except Exception as e:
                print(e)
                
        return weights_optimized

In [106]:
port = PortfolioOptimizerTest()

In [107]:
port.set_initial_asset(['Asset1', 'Asset2'], [0.6, 0.4])


Initial weights sum: 1.0


In [108]:
port.cvar_value = 0.24
port.cvar_value

0.015118578920369087

In [99]:
port.risk_level = 'Very Aggressive'

In [100]:
port.obj

'MaxRet'

In [101]:
port.port_optimize(Y)


,weights
Asset1,60.0000%
Asset2,40.0000%
APA,0.0000%
BA,0.0000%
BAX,0.0000%
BMY,0.0000%
CMCSA,0.0000%
CNP,0.0000%
CPB,0.0000%
DE,0.0000%
